In [1]:
import re
import glob
import csv
import os
import shutil
import copy
from collections import Counter
import pandas as pd

def titlecase(s):
    return re.sub(
        r"[A-Za-z]+('[A-Za-z]+)?",
        lambda word: word.group(0).capitalize(),
        s)

In [2]:
class stickerCollection:
    def __init__(self):
        #a-z,A-Z,1-0
        self.stickerDF = {}
        path = "*.csv"
        print("today's sticker options")
        for fname in glob.glob(path):
            with open(fname, mode='r') as inp:
                reader = csv.reader(inp)
                dict_from_csv = {rows[0]:rows[1] for rows in reader}
                dict_from_csv = dict([a, int(x)] for a, x in dict_from_csv.items())
                if (fname == "template.csv"):
                    self.initialValues =  dict_from_csv
                else:
                    print(fname[:-4])
                    self.stickerDF[fname[:-4]] = dict_from_csv
                    
    def addSet(self,name,init):
        newSet = self.initialValues.copy()
        self.stickerDF[name] = newSet
        for key in init:
            newSet[str(key)] = init[key]
        with open(name+".csv", 'w') as f:
            for key in newSet:
                f.write("%s,%s\n"%(key,newSet[key]))
                
    def search(self,message):
        lowercaseCounter = dict(Counter(message.lower()))
        uppercaseCounter = dict(Counter(message.upper()))
        propercaseCounter = dict(Counter(titlecase(message)))
        output = pd.DataFrame(columns=['Sticker Set', 'Uppercase', 'Lowercase',"Propercase","numbers"])
        for keyStickers in self.stickerDF:
            currentStickerSet = self.stickerDF[keyStickers];
            results = [keyStickers,True,True,True,True]
            for key in lowercaseCounter:
                if key == " ":
                    continue
                if key in ["0","1","2","3","4","5","6","7","8","9"]:
                    if lowercaseCounter[key] > currentStickerSet[key]:
                        results[4] = False  
                else:
                    if lowercaseCounter[key] > currentStickerSet[key]:
                        results[2] = False
            for key in uppercaseCounter:
                if key == " ":
                    continue
                if key not in ["0","1","2","3","4","5","6","7","8","9"]:
                    if uppercaseCounter[key] > currentStickerSet[key]:
                        results[1] = False
            for key in propercaseCounter:
                if key == " ":
                    continue
                if key not in ["0","1","2","3","4","5","6","7","8","9"]:
                    if propercaseCounter[key] > currentStickerSet[key]:
                        results[3] = False
            output.loc[len(output.index)] = results
        return output
    def remove(self,message,setName,case):
        if (case.lower() == "upper"):
            counter = dict(Counter(message.upper()))
        elif (case.lower() == "lower"):
            counter = dict(Counter(message.lower()))
        elif (case.lower() == "proper"):
            counter = dict(Counter(titlecase(message)))
        currentStickerSet = self.stickerDF[setName];
        for key in counter:
            if key == " ":
                continue
            currentStickerSet[key] = currentStickerSet[key] - counter[key]
        with open(setName+".csv", 'w') as f:
            for key in currentStickerSet:
                f.write("%s,%s\n"%(key,currentStickerSet[key]))

In [3]:
collection = stickerCollection()

today's sticker options
gold
silver
shinnyred1
golddisco
goldwithblackoutline
goldCircle
burgundy


In [5]:
collection.search("slovenia 2022")

,Sticker Set,Uppercase,Lowercase,Propercase,numbers
0,gold,False,False,False,False
1,silver,False,False,False,False
2,shinnyred1,True,True,True,False
3,golddisco,True,True,True,False
4,goldwithblackoutline,True,False,False,True
5,goldCircle,False,False,False,False
6,burgundy,True,True,True,False


In [19]:
collection.remove("july","gold","upper")